In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

# 0. 모듈 & 라이브러리

In [56]:
import sys 
import os  

import pandas as pd  
import numpy as np
import re

from konlpy.tag import Okt
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import GridSearchCV  


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# !pip install konlpy soynlp : 필요시 soynlp 설치

# 1. 데이터 확인

In [2]:
pwd()

'c:\\Users\\Jay\\Desktop\\dacon\\movie_review'

In [3]:
# 인코딩 후 저장
df = pd.read_csv('./dataset/train.csv', encoding='utf-8-sig')
df.to_csv('train_enc.csv', encoding = 'utf-8-sig')

In [5]:
# train 데이터 불러오기
df = pd.read_csv('./dataset/train_enc.csv', encoding='utf-8-sig')
df = df.drop("Unnamed: 0", axis=1)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5000 non-null   int64 
 1   document  5000 non-null   object
 2   label     5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


In [36]:
df.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [37]:
# 결측값 확인
df.isna().sum()

id          0
document    0
label       0
dtype: int64

In [31]:
# 긍정/부정 데이터 셋 확인
df.groupby('label').size()

label
0    2564
1    2436
dtype: int64

In [35]:
df[df['label']==0]

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0
6,7,성룡영화중 최악 .. 차라리 러시아워가 훨씬 나음,0
9,10,결말 기분나쁘다 공포매니아에게 이따위 결말을,0
...,...,...,...
4994,4995,도대체 누가 안소니짐머라는거지?,0
4995,4996,좋은 배우들로 3류영화를 찍은 안타까운 영화,0
4996,4997,진짜 드럽게 재미없다 에드워드 호퍼 그림에 배경 빼고는 볼게 아닌영화,0
4997,4998,가장 실망스러운 영화.. 지금까지 본영화중..,0


# 2. 데이터 전처리

In [47]:
#id 컬럼 제거
df.drop('id',axis=1, inplace=True)

In [48]:
# 전처리 함수 정의 : 정규 표현식 이용
def preprocessor(text): 
    text = re.sub('[^가-힣\s]+','',text) # 한글, 공백문자 남기고 다 날리기
    return text

In [49]:
preprocessor('개재밌음 ㅋㅋㅋ omg @@@ !! ^')


'개재밌음     '

In [50]:
# 전처리 함수
list_clean = []
for i in range(0,len(df['document'])):
    df['document'][i]=preprocessor(df['document'][i])

In [51]:
df['document']

0                    영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐
1                    히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯
2             괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요
3                    아무래도 년도지난작품이라 지금보기는너무유치하다
4        지금까지의 영화들이 그랬듯 이 영화역시 일본에 대한 미화는 여전하다
                         ...                  
4995                   좋은 배우들로 류영화를 찍은 안타까운 영화
4996    진짜 드럽게 재미없다 에드워드 호퍼 그림에 배경 빼고는 볼게 아닌영화
4997                     가장 실망스러운 영화 지금까지 본영화중
4998             이런 평점 테러 네이버에서 좀 막아야 하는 것 아닌가
4999         여주인공이 전작에서는 귀여웠는데 여기선 완전 망했네 실망이다
Name: document, Length: 5000, dtype: object

In [52]:
# 전처리 완료 함수 저장
df.to_csv('dacon_clean.csv', encoding='utf-8-sig')

In [53]:
df.head()

,document,label
0,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯,1
2,괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요,1
3,아무래도 년도지난작품이라 지금보기는너무유치하다,0
4,지금까지의 영화들이 그랬듯 이 영화역시 일본에 대한 미화는 여전하다,0


In [54]:
# 결측치 다시 확인
df.isna().sum()

document    0
label       0
dtype: int64

# 3. TFIDF VECTORIZE

In [58]:
X_train = df['document'].values
y_train = df['label'].values

In [73]:
test = pd.read_csv('dataset/test.csv', encoding='utf-8-sig')
print(test.shape)
test.head(3)

(5000, 2)


,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy


In [74]:
X_test = test['document']
print(X_test.shape)
X_test.head()

(5000,)


0                       시간 때우기 좋은 영화 지루함
1    훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2                      Childhood fantasy
3                    멋있는 영화입니다. 잊을 수 없는!
4                     너무 감동적이네요 펑펑 울었습니다
Name: document, dtype: object

In [59]:
# okt 형태소 분석기 사용
okt = Okt()

In [61]:
print(X_train[0])

영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐


In [62]:
print(okt.morphs(X_train[0]))

['영상', '이나', '음악', '이', '이쁘다', '해도', '미화', '시킨', '불륜', '일', '뿐']


In [64]:
# tfidf 적용
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.90,tokenizer=okt.morphs, token_pattern=None) 
# 1or2 단어 뭉치, 3회 미만 제거, 상위 10% 단어 제거, 토크나이저 okt뭉치, 정규표현 미적용
tfidf.fit(X_train)

TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2), token_pattern=None,
                tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x0000019BD551D910>>)

In [75]:
X_train_okt = tfidf.transform(X_train)
X_test_okt = tfidf.transform(X_test)

# 4. SGD Classifier 

In [66]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.utils.fixes import loguniform
from sklearn.metrics import accuracy_score

In [69]:
# SGD Classifier 객체생성 / 손실함수 : 로그함수
sgd = SGDClassifier(loss='log', random_state=1)

In [67]:
# alpha 규제값 탐색 범위
param_dist = {'alpha': loguniform(0.0001,100.0)}

In [76]:
# randomizedSearchCV 
rsv_okt = RandomizedSearchCV(estimator=sgd, param_distributions=param_dist, n_iter=300, random_state=1, verbose=1)
rsv_okt.fit(X_train_okt, y_train)
preds = rsv_okt.predict(X_train_okt)
accuracy = accuracy_score(y_train, preds)
print ("accuracy: ",accuracy)
print(rsv_okt.best_score_)
print(rsv_okt.best_params_)

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
accuracy:  0.9374
0.8453999999999999
{'alpha': 0.00011894217640347858}


In [77]:
# 타겟 생성
predsTest = rsv_okt.predict(X_test_okt)

In [78]:
len(predsTest)

5000

In [79]:
# 제출 파일에 타깃 저장
submission = pd.read_csv("dataset/sample_submission.csv")
submission["label"] = predsTest
print(submission.shape)
submission.head()

(5000, 2)


,id,label
0,1,0
1,2,1
2,3,1
3,4,1
4,5,1


In [80]:
# 제출 파일 저장
submission.to_csv("dataset/Accuracy_{0:.5f}_submission.csv".format(accuracy), index=False)